# Parse Txt

In [1]:
import PyPDF2 as pydf
import pandas as pd
from pandas import DataFrame as DF, Series
import numpy as np
import re

In [2]:
path = 'results-5-2010.pdf'
pdf = open(path, 'rb')

In [3]:
reader = pydf.PdfFileReader(pdf)

In [4]:
page = reader.getPage(1)

In [5]:
print(page.extractText())

May 18, 2010, Primary Election Abstracts of Votes * Nominee 
** Elected WI = Write In
 2UNITED STATES SENATOR Republican  Stutzman Parker Later Dinkel 
*Huffman Woodland Waldron Misc. 
County Tom Robin S Loren G Shane Jim Walter H       Keith  
Baker 235 106 375 364 587 32 245 18 
Benton 797 268 919 702 2,207 86 431 51 
Clackamas 2,940 1,281 3,590
 3,054 13,624 382 2,582 162 
Clatsop 345 199 302 276 1,044 45 259 45 
Columbia 383 171 485 568 1,555 66 238 43 
Coos 843 466 979 830 1,773 128 598 56 

Crook 255 122 329 441 894 60 332 28 
Curry 339 188 516 539 746 55 258 33 
Deschutes 1,053 813 2,161 2,103 6,677 189 1,264 107 
Douglas 1,362 652 1,983 1,901 4,784 208 1,314 113 
Gilliam 20 12 22 28 171 0 11 1 

Grant 90 41 93 209 590 18 81 14 
Harney 147 71 184 212 280 15 153 19 
Hood River 201 104 148 164 574 21 122 11 
Jackson 1,429 925 3,549 2,838 5,250 251 1,603 106 
Jefferson 116 67 144 297 1,168 34 121  9 

Josephine 890 570 1,568 1,661 2,535 192 1,075 102 
Klamath 640 501 1,419 1,236 1,

In [8]:
pages = []
for i in range(reader.numPages):
    pages.append(reader.getPage(i).extractText())

In [9]:
[m.span()[1] for m in re.finditer('County', pages[2])]

[165, 438]

In [10]:
print(pages[-1])

May 18, 2010, Primary Election Abstracts of Votes  * Indicates Passage or Nonpassage of Measure  
 69 STATE MEASURE NO. 69  Amends Constitution: Continues And Modernizes Authority For Lowest Cost Borrowing For Community Colleges 
And Public Universities. 
 County *Yes No 
 Baker 2,511 1,612 
Benton 15,424 3,825 
Clackamas 50,652 18,982 
Clatsop 6,935 2,532 

Columbia 7,358 3,579 
Coos 10,092 6,045 
Crook 3,059 2,071 
Curry 4,228 2,540 
Deschutes 25,093 11,889 

Douglas 15,407 10,217 
Gilliam 436 181 
Grant 1,139 825 
Harney 1,238 906 
Hood River 4,246 1,539 

Jackson 26,102 11,495 
Jefferson 2,721 1,527 
Josephine 13,168 6,367 
Klamath 9,643 6,012 
Lake 1,212 962 

Lane 59,630 22,301 
Lincoln 8,264 2,924 
Linn 14,705 8,038 
Malheur 3,682 1,878 
Marion 37,681 19,367 

Morrow 1,474 778 
Multnomah 105,000 21,753 
Polk 11,713 5,007 
Sherman 362 222 
Tillamook 4,765 1,965 

Umatilla 7,570 3,861 
Union 4,000 2,128 
Wallowa 1,519 832 
Wasco 4,017 1,828 
Washington 66,788 23,129 

Wheeler 248 

In [11]:
def get_table_starts(p):
    """ Return list of string indexes where tables begin
        on given page (p).
    """
    return [m.start() for m in re.finditer(r'County', pages[p])]

In [12]:
def get_column_lines(p, table_starts):
    starts = [m.start() for m in re.finditer('Republican|Democrat', pages[p])]
    n = len(starts)
    lines = [pages[p][starts[i]:table_starts[i]].strip() if i < n-1
            else pages[p][starts[i]:table_starts[i]].strip() for i in range(n)]
#     return lines
    all_cols = [re.search('(?<=Republican).*|(?<=Democrat).*', l).group().strip().split() for l in lines]
    return [[c for c in cols if c not in ['Jr', 'Sr', 'III', 'IV', 'V', 'VI']]
            for cols in all_cols]

In [13]:
get_column_lines(29, get_table_starts(29))

[['*Clem', 'Misc.'],
 ['*Sannes', 'Alamarez', 'Misc.'],
 ['*Komp', 'Misc.'],
 ['*LeCompte', 'Chereck', 'Misc.']]

In [14]:
def get_table_titles(p):
    titles = [l for l in pages[p].split('\n') if re.search('Republican|Democrat', l)]
    spans = []
    for t in titles:
        try:
            spans.append(re.search('\d+', t.strip()[:3]).span())
        except:
            spans.append(None)
    titles = [t.strip()[s[1]:] if s else t for s,t in zip(spans, titles)]
    return [re.match('(.*?)Democrat|(.*?)Republican', t)\
            .group().strip()\
            .replace(',', '')\
            .replace(' ', '_') for t in titles]

In [15]:
get_table_titles(2)

['REPRESENTATIVE_IN_CONGRESS_1ST_DISTRICT_Democrat',
 'REPRESENTATIVE_IN_CONGRESS_1ST_DISTRICT_Republican']

In [16]:
def get_table_strings(p, idxs):
    n = len(idxs)
    return [pages[p][idxs[i]:idxs[i+1]-1].lstrip() if i < n-1
            else pages[p][idxs[i]:].lstrip() for i in range(n)]

In [17]:
def parse_tables(p):
    idxs = get_table_starts(p)
    strings = get_table_strings(p, idxs)
    all_columns = get_column_lines(p, idxs)
    dfs = []
    # iterate over table strings on current page (p)
    for s,cols in zip(strings, all_columns):
        # convert table string to line strings
        lines = [l.strip() for l in re.sub('\n\s*(?=\d)', ' ', s).split('\n') if l != '']
        lines = [re.sub('\s{2,}', ' ', l) for l in lines]
        # get first and last row line numbers
        first_row_line = [i for i,l in enumerate(lines) if l.startswith('County')][0] + 1
        last_row_line = [i for i,l in enumerate(lines) if l.startswith('TOTAL')][0]
        # parse rows from line strings
        rows = [re.sub('(?!\d), (?=\d)', '', r) for r in lines[first_row_line:last_row_line+1]]
        rows = [re.sub('\s+(?=\D)', '-', l).replace(',', '').split(' ') for l in rows]
        # convert numeric elements to int
        rows = [[int(el) if el.isdigit() else el for el in r] for r in rows]
        dfs.append(DF(rows, columns=['County'] + cols))
    return dfs

In [18]:
dfs = parse_tables(2)

In [19]:
from IPython.display import display
for df in dfs:
    display(df)

,County,Robinson,*Wu,Misc.
0,Clatsop,888,3937,25
1,Columbia,1510,4495,46
2,Multnomah,1812,11843,31
3,Washington,8029,34593,201
4,Yamhill,1863,6571,80
5,TOTAL,14102,61439,383


,County,Keller,Kuzmanich,Brodhead,*Cornilles,Misc.
0,Clatsop,732,681,94,1146,37
1,Columbia,1110,1146,121,1091,23
2,Multnomah,776,971,50,1388,26
3,Washington,9886,9070,725,13764,163
4,Yamhill,2281,2596,223,4052,50
5,TOTAL,14785,14464,1213,21441,299
